# Grid Independence Test

A benchmarking workflow to verify that CFD results are **mesh-independent**.

The experiment fixes **one terrain** and **one wind direction** and runs the same
simulation with a family of progressively finer mesh configurations. If the
flow statistics (e.g. velocity profiles, turbulence quantities) converge as the
mesh is refined, the coarsest mesh that achieves the desired accuracy level
can be selected for the full dataset generation run.

**Workflow overview:**
1. For each mesh variant, copy the fixed terrain inputs and write a modified
   `terrain_config.yaml` with the variant-specific grid / mesh overrides.
2. Run the terrain mesh pipeline (or skip if already done) to produce OpenFOAM
   case inputs — one per variant.
3. Generate OpenFOAM cases and mesh them locally (or on HPC).
4. Submit all variants to SLURM and monitor their progress.
5. Compare key metrics across variants to establish mesh independence.

**Resume-safe:** Close and reopen at any time.  
All decisions are derived from `benchmark_metadata.json` files written into each
variant directory.

## 1. Configuration

Edit these settings before running the notebook.

In [ ]:
import os
import sys

# ── Paths ────────────────────────────────────────────────────────────────────
# Root of the CFD-dataset repository (directory containing this notebook)
REPO_ROOT = os.path.dirname(os.path.abspath("__file__"))

# Path to the single terrain directory to use for this benchmark
# (e.g. a folder produced by generateInputs.py under Data/downloads/)
FIXED_TERRAIN_DIR = os.path.join(REPO_ROOT, "Data", "downloads", "terrain_0001_example")

# Single wind direction to test (degrees, 0 = North)
FIXED_ROTATION_DEG = 270

# Base terrain configuration file — variant overrides are layered on top of this
TERRAIN_CONFIG_PATH = os.path.join(REPO_ROOT, "terrain_config.yaml")

# Root output directory: one sub-folder will be created per mesh variant
CASES_OUTPUT_DIR = os.path.join(REPO_ROOT, "grid_independence_benchmark")

# Path to the taskManager submodule
TASK_MANAGER_DIR = os.path.join(REPO_ROOT, "taskManager")

# Remote HPC path on Deucalion (used by taskManager for rsync/sbatch)
DEUCALION_PATH = "/projects/EEHPC-BEN-2026B02-011/cfd_data"

# Number of parallel workers for local meshing
N_PARALLEL_WORKERS = 4

# ── Mesh variants ─────────────────────────────────────────────────────────────
# Each entry must have a unique "name" key.  All other keys are deep-merged
# into the base terrain_config.yaml before running the mesh pipeline.
# Use the same top-level section names as terrain_config.yaml
# ("grid", "mesh", "terrain", "boundary", …).
MESH_VARIANTS = [
    {
        "name": "coarse",
        "grid": {"nx": 181, "ny": 181},
        "mesh": {"total_z_cells": 44},
    },
    {
        "name": "medium",
        "grid": {"nx": 271, "ny": 271},
        "mesh": {"total_z_cells": 55},
    },
    {
        "name": "fine",  # matches the baseline in terrain_config.yaml
        "grid": {"nx": 361, "ny": 361},
        "mesh": {"total_z_cells": 66},
    },
    {
        "name": "very_fine",
        "grid": {"nx": 451, "ny": 451},
        "mesh": {"total_z_cells": 80},
    },
]

# ── SLURM settings ────────────────────────────────────────────────────────────
# These are applied to every submitted job (all variants use the same resources)
SLURM_PARTITION        = "hpc"
SLURM_TIME             = "04:00:00"   # wall-clock time limit per job
SLURM_N_NODES          = 2
SLURM_NTASKS_PER_NODE  = 128

# ── Submodule path setup ──────────────────────────────────────────────────────
for _submod in ["terrain_following_mesh_generator", "ABL_BC_generator", TASK_MANAGER_DIR]:
    _p = _submod if os.path.isabs(_submod) else os.path.join(REPO_ROOT, _submod)
    if _p not in sys.path:
        sys.path.insert(0, _p)

print(f"REPO_ROOT           : {REPO_ROOT}")
print(f"FIXED_TERRAIN_DIR   : {FIXED_TERRAIN_DIR}")
print(f"FIXED_ROTATION_DEG  : {FIXED_ROTATION_DEG}")
print(f"TERRAIN_CONFIG_PATH : {TERRAIN_CONFIG_PATH}")
print(f"CASES_OUTPUT_DIR    : {CASES_OUTPUT_DIR}")
print(f"N_PARALLEL_WORKERS  : {N_PARALLEL_WORKERS}")
print(f"Mesh variants       : {[v['name'] for v in MESH_VARIANTS]}")

## 2. Imports

In [ ]:
import copy
import json
from pathlib import Path
from datetime import datetime

import yaml
import pandas as pd

# ── terrain_following_mesh_generator (submodule) ──────────────────────────────
try:
    from terrain_following_mesh_generator import terrain_mesh as tm
    _MESH_OK = True
    print("✓ terrain_following_mesh_generator imported")
except ImportError as _e:
    _MESH_OK = False
    print(f"✗ terrain_following_mesh_generator not available: {_e}")
    print("  Run: git submodule update --init --recursive")

# ── taskManager (submodule) ───────────────────────────────────────────────────
try:
    from taskManager import OpenFOAMCaseGenerator
    _TM_OK = True
    print("✓ taskManager imported")
except ImportError as _e:
    _TM_OK = False
    OpenFOAMCaseGenerator = None
    print(f"✗ taskManager not available: {_e}")
    print("  Run: git submodule update --init --recursive")

## 3. Generate Mesh Variant Inputs

For each entry in `MESH_VARIANTS`:
1. Create a variant output directory (`grid_bench_{name}_{rotation:03d}deg/`).
2. Load `terrain_config.yaml` and deep-merge the variant overrides.
3. Save the modified config to a `variant_terrain_config.yaml` inside the variant dir.
4. Run the terrain mesh pipeline with the modified config (if the submodule is available).
5. Write a `benchmark_metadata.json` to record variant parameters and pipeline status.

Variants that already have a `benchmark_metadata.json` with `status == "complete"`
are **skipped** (resume-safe).

In [ ]:
def _deep_merge(base: dict, overrides: dict) -> dict:
    """Recursively merge *overrides* into a deep copy of *base*."""
    result = copy.deepcopy(base)
    for key, value in overrides.items():
        if key == "name":
            continue  # skip the variant name key
        if isinstance(value, dict) and isinstance(result.get(key), dict):
            result[key] = _deep_merge(result[key], value)
        else:
            result[key] = copy.deepcopy(value)
    return result


def _variant_dir(cases_output_dir: str, name: str, rotation: int) -> Path:
    return Path(cases_output_dir) / f"grid_bench_{name}_{rotation:03d}deg"


def _mesh_params_summary(variant: dict) -> str:
    """One-liner summary of the mesh parameters for display in tables."""
    parts = []
    if "grid" in variant:
        g = variant["grid"]
        if "nx" in g and "ny" in g:
            parts.append(f"nx={g['nx']}, ny={g['ny']}")
    if "mesh" in variant:
        m = variant["mesh"]
        if "total_z_cells" in m:
            parts.append(f"nz={m['total_z_cells']}")
    return "; ".join(parts) if parts else str(variant)


# ── Load base config ──────────────────────────────────────────────────────────
with open(TERRAIN_CONFIG_PATH) as fh:
    base_config = yaml.safe_load(fh)
print(f"✓ Loaded base config: {TERRAIN_CONFIG_PATH}")

# ── Locate terrain inputs (DEM / roughness) ───────────────────────────────────
terrain_path = Path(FIXED_TERRAIN_DIR)
if not terrain_path.exists():
    print(f"✗ FIXED_TERRAIN_DIR not found: {FIXED_TERRAIN_DIR}")
    print("  Set FIXED_TERRAIN_DIR to an existing terrain directory and re-run.")
else:
    dem_files  = sorted([f for f in terrain_path.glob("terrain_*.tif") if "_raw" not in f.name])
    rmap_files = sorted([f for f in terrain_path.glob("roughness_*.tif") if "_raw" not in f.name])
    dem_file       = str(dem_files[0])  if dem_files  else None
    roughness_file = str(rmap_files[0]) if rmap_files else None
    print(f"✓ Terrain dir    : {terrain_path.name}")
    print(f"  DEM file       : {Path(dem_file).name if dem_file else 'NOT FOUND'}")
    print(f"  Roughness file : {Path(roughness_file).name if roughness_file else 'not found'}")

# ── Generate variant inputs ───────────────────────────────────────────────────
Path(CASES_OUTPUT_DIR).mkdir(parents=True, exist_ok=True)

for variant in MESH_VARIANTS:
    name       = variant["name"]
    var_dir    = _variant_dir(CASES_OUTPUT_DIR, name, FIXED_ROTATION_DEG)
    meta_file  = var_dir / "benchmark_metadata.json"

    # ── Resume check ─────────────────────────────────────────────────────────
    if meta_file.exists():
        with open(meta_file) as fh:
            meta = json.load(fh)
        if meta.get("status") == "complete":
            print(f"  ↷ {name}: already complete, skipping")
            continue

    var_dir.mkdir(parents=True, exist_ok=True)

    # ── Build modified config ─────────────────────────────────────────────────
    variant_config = _deep_merge(base_config, variant)
    variant_config.setdefault("terrain", {})
    variant_config["terrain"]["rotation_deg"] = FIXED_ROTATION_DEG

    variant_config_path = var_dir / "variant_terrain_config.yaml"
    with open(variant_config_path, "w") as fh:
        yaml.dump(variant_config, fh, default_flow_style=False, sort_keys=False)

    # ── Initial metadata ──────────────────────────────────────────────────────
    meta = {
        "name"        : name,
        "mesh_params" : _mesh_params_summary(variant),
        "terrain_dir" : str(FIXED_TERRAIN_DIR),
        "rotation"    : FIXED_ROTATION_DEG,
        "status"      : "pending",
        "created_at"  : datetime.now().isoformat(),
    }
    with open(meta_file, "w") as fh:
        json.dump(meta, fh, indent=2)

    # ── Run mesh pipeline ─────────────────────────────────────────────────────
    if not _MESH_OK:
        print(f"  ○ {name}: mesh pipeline not available (submodule missing)")
        continue

    if not terrain_path.exists() or dem_file is None:
        print(f"  ✗ {name}: FIXED_TERRAIN_DIR missing or has no DEM — skipping mesh step")
        continue

    print(f"  ▶ {name}: running mesh pipeline…")
    try:
        mesh_config = tm.load_config(str(variant_config_path))
        pipeline    = tm.TerrainMeshPipeline()
        pipeline.run(
            dem_path=dem_file,
            rmap_path=roughness_file,
            output_dir=str(var_dir),
            **mesh_config,
        )
        meta["status"] = "complete"
        print(f"  ✓ {name}: mesh pipeline complete")
    except Exception as exc:
        meta["status"] = "failed"
        meta["error"]  = str(exc)
        print(f"  ✗ {name}: {exc}")

    meta["updated_at"] = datetime.now().isoformat()
    with open(meta_file, "w") as fh:
        json.dump(meta, fh, indent=2)

print("\nVariant input generation complete.")

## 4. Status Scanner

Reads every `benchmark_metadata.json` found under `CASES_OUTPUT_DIR` and assembles a
pandas DataFrame.  **Re-run this cell at any time to refresh the view.**

In [ ]:
def scan_benchmark_status(cases_output_dir: str) -> pd.DataFrame:
    """Scan variant directories and return a status DataFrame."""
    output_path = Path(cases_output_dir)
    records = []

    if not output_path.exists():
        print(f"⚠  Output directory does not exist yet: {cases_output_dir}")
        print("   Run Section 3 first to generate variant inputs.")
    else:
        for meta_file in sorted(output_path.glob("grid_bench_*/benchmark_metadata.json")):
            try:
                with open(meta_file) as fh:
                    meta = json.load(fh)
            except (json.JSONDecodeError, OSError) as exc:
                print(f"⚠  Could not read {meta_file}: {exc}")
                continue

            # Check for a sibling case_status.json written by taskManager
            case_status_file = meta_file.parent / "case_status.json"
            case_status      = {}
            if case_status_file.exists():
                try:
                    with open(case_status_file) as fh:
                        case_status = json.load(fh)
                except (json.JSONDecodeError, OSError):
                    pass

            records.append({
                "variant_name"    : meta.get("name"),
                "mesh_params"     : meta.get("mesh_params"),
                "pipeline_status" : meta.get("status"),
                "mesh_status"     : case_status.get("mesh_status"),
                "job_id"          : case_status.get("job_id"),
                "job_status"      : case_status.get("job_status"),
                "last_checked"    : case_status.get("last_checked"),
                "variant_dir"     : str(meta_file.parent),
            })

    columns = [
        "variant_name", "mesh_params", "pipeline_status",
        "mesh_status", "job_id", "job_status", "last_checked", "variant_dir",
    ]
    return pd.DataFrame(records, columns=columns) if records else pd.DataFrame(columns=columns)


df_bench = scan_benchmark_status(CASES_OUTPUT_DIR)
print(f"Scanned {len(df_bench)} variant(s) at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## 5. Summary Dashboard

In [ ]:
STATUS_ICONS = {
    "complete"      : "★",
    "pending"       : "○",
    "failed"        : "✗",
    "ready_to_mesh" : "○",
    "meshing"       : "⟳",
    "meshed"        : "✓",
    "running"       : "▶",
}

total = len(df_bench)
print(f"{'='*55}")
print(f"  GRID INDEPENDENCE TEST — STATUS SUMMARY")
print(f"  Terrain  : {Path(FIXED_TERRAIN_DIR).name}")
print(f"  Rotation : {FIXED_ROTATION_DEG}°")
print(f"  {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"{'='*55}")
print(f"  Total variants : {total}")
print()
if total > 0:
    counts = df_bench["pipeline_status"].value_counts()
    for status, n in counts.items():
        icon = STATUS_ICONS.get(str(status), "?")
        print(f"  {icon}  {str(status):<16} : {n}")
print(f"{'='*55}")

In [ ]:
# ── Full variant table ────────────────────────────────────────────────────────
if not df_bench.empty:
    display(
        df_bench[
            ["variant_name", "mesh_params", "pipeline_status",
             "mesh_status", "job_id", "job_status", "last_checked"]
        ].reset_index(drop=True)
    )
else:
    print("No variants found. Run Section 3 first.")

## 6. Generate & Mesh Cases

Uses `taskManager` to convert each variant input directory into a full OpenFOAM case
and then mesh it locally using `blockMesh` + `snappyHexMesh`.

**Resume-safe:** variants that already have `mesh_status == DONE` in their
`case_status.json` are skipped.

In [ ]:
if not _TM_OK:
    print("✗ taskManager not available — cannot generate or mesh cases.")
    print("  Run: git submodule update --init --recursive")
else:
    # ── Initialise the case generator ─────────────────────────────────────────
    generator = OpenFOAMCaseGenerator(
        template_path=os.path.join(TASK_MANAGER_DIR, "template"),
        input_dir=CASES_OUTPUT_DIR,
        output_dir=CASES_OUTPUT_DIR,
        deucalion_path=DEUCALION_PATH,
    )

    # Re-scan to pick up the latest state
    df_bench = scan_benchmark_status(CASES_OUTPUT_DIR)

    # ── Generate cases for variants that have completed mesh pipeline input ────
    for _, row in df_bench.iterrows():
        name      = row["variant_name"]
        var_dir   = Path(row["variant_dir"])
        cs_file   = var_dir / "case_status.json"

        if cs_file.exists():
            with open(cs_file) as fh:
                cs = json.load(fh)
            if (cs.get("mesh_status") or "").upper() in ("DONE", "IN_PROGRESS"):
                print(f"  ↷ {name}: already meshed / meshing, skipping")
                continue

        if row["pipeline_status"] != "complete":
            print(f"  ○ {name}: pipeline_status={row['pipeline_status']}, skipping case generation")
            continue

        try:
            output_path = generator.setup_case({"variant_dir": str(var_dir), "name": name})
            print(f"  ✓ {name}: case created at {output_path}")
        except Exception as exc:
            print(f"  ✗ {name}: case setup failed: {exc}")

    # ── Mesh cases that are ready ──────────────────────────────────────────────
    ready_cases = [
        str(Path(row["variant_dir"]))
        for _, row in df_bench.iterrows()
        if row["mesh_status"] in (None, "NOT_RUN", "")
        and row["pipeline_status"] == "complete"
    ]

    if not ready_cases:
        print("\nNo cases ready for meshing.")
    else:
        print(f"\nMeshing {len(ready_cases)} case(s) with {N_PARALLEL_WORKERS} worker(s)…")
        results = generator.mesh_cases_parallel(ready_cases, n_workers=N_PARALLEL_WORKERS)
        succeeded = sum(results)
        failed    = len(results) - succeeded
        print(f"Meshing complete: {succeeded} succeeded, {failed} failed.")
        print("Re-run Section 4 to refresh the dashboard.")

## 7. Submit to HPC

Submits all meshed (but not yet submitted) variants to SLURM via the taskManager.
All variants use the same SLURM resource settings defined in Section 1.

In [ ]:
if not _TM_OK:
    print("✗ taskManager not available — cannot submit jobs.")
else:
    df_bench = scan_benchmark_status(CASES_OUTPUT_DIR)

    slurm_config = {
        "partition"       : SLURM_PARTITION,
        "time"            : SLURM_TIME,
        "nodes"           : SLURM_N_NODES,
        "ntasks_per_node" : SLURM_NTASKS_PER_NODE,
    }

    meshed_cases = df_bench[
        (df_bench["mesh_status"] == "DONE") &
        (df_bench["job_id"].isna())
    ]

    if meshed_cases.empty:
        print("No meshed cases ready for submission.")
        print("Possible reasons:")
        print("  • Meshing has not finished yet — re-run Section 6.")
        print("  • All variants have already been submitted.")
    else:
        print(f"Submitting {len(meshed_cases)} variant(s) to SLURM…")
        for _, row in meshed_cases.iterrows():
            name     = row["variant_name"]
            var_path = row["variant_dir"]
            try:
                job_id = generator.submit_case(
                    case_path=var_path,
                    **slurm_config,
                )
                print(f"  ✓ {name}: submitted (job_id={job_id})")
            except Exception as exc:
                print(f"  ✗ {name}: submission failed: {exc}")

        print("\nSubmission complete. Re-run Section 4 to refresh the dashboard.")

## 8. Refresh Job Statuses

Polls the SLURM scheduler for every submitted variant and writes the updated status
back to each `case_status.json`.  Requires SSH access to the `deucalion` host.

In [ ]:
if not _TM_OK:
    print("✗ taskManager not available.")
else:
    df_bench = scan_benchmark_status(CASES_OUTPUT_DIR)
    submitted = df_bench[df_bench["job_id"].notna()]

    if submitted.empty:
        print("No submitted jobs to refresh.")
    else:
        print(f"Refreshing {len(submitted)} job status(es)…")
        for _, row in submitted.iterrows():
            try:
                new_status = generator.update_job_status(row["variant_dir"])
                print(f"  {row['variant_name']}: {new_status}")
            except Exception as exc:
                print(f"  ✗ {row['variant_name']}: {exc}")

        print("\nJob statuses updated. Re-run Section 4 to refresh the dashboard.")

## 9. Results Summary

Once all variants have completed, compare the key metrics across mesh resolutions
to assess grid independence:

| Metric | Source |
|---|---|
| Total cell count | `constant/polyMesh/owner` or log files |
| Wall-clock time | SLURM job accounting (`sacct`) |
| Convergence indicator | Final residual from `log.simpleFoam` |

Re-run this cell after all jobs have reached `COMPLETED` status.

In [ ]:
import re

def _parse_cell_count(case_path: Path) -> int | None:
    """Try to read total cell count from blockMesh or checkMesh log."""
    for log_name in ("log.blockMesh", "log.snappyHexMesh", "log.checkMesh"):
        log_file = case_path / log_name
        if not log_file.exists():
            continue
        text = log_file.read_text(errors="replace")
        # Look for "cells:" line
        m = re.search(r"cells:\s+(\d+)", text)
        if m:
            return int(m.group(1))
    return None


def _parse_final_residual(case_path: Path) -> float | None:
    """Extract the final Ux residual from log.simpleFoam (last occurrence)."""
    log_file = case_path / "log.simpleFoam"
    if not log_file.exists():
        return None
    text = log_file.read_text(errors="replace")
    # Pattern: "Solving for Ux, Initial residual = X, Final residual = Y"
    matches = re.findall(r"Solving for Ux.*?Final residual = ([0-9eE.+-]+)", text)
    if matches:
        try:
            return float(matches[-1])
        except ValueError:
            return None
    return None


# ── Build results table ───────────────────────────────────────────────────────
df_bench = scan_benchmark_status(CASES_OUTPUT_DIR)

results = []
for _, row in df_bench.iterrows():
    var_path    = Path(row["variant_dir"])
    cell_count  = _parse_cell_count(var_path)
    final_resid = _parse_final_residual(var_path)
    results.append({
        "variant_name"   : row["variant_name"],
        "mesh_params"    : row["mesh_params"],
        "job_status"     : row["job_status"],
        "total_cells"    : cell_count,
        "final_residual" : final_resid,
        "wall_time"      : None,   # populate from sacct if available
    })

df_results = pd.DataFrame(results)

if df_results.empty:
    print("No results yet. Wait for all jobs to complete and re-run this cell.")
else:
    print("Grid independence results:")
    display(df_results.reset_index(drop=True))

    completed = df_results[df_results["job_status"] == "COMPLETED"]
    if len(completed) < 2:
        print("\n⚠  Fewer than 2 variants completed — cannot yet assess independence.")
    else:
        # Simple check: relative change in final residual between successive refinements
        print("\nRelative change in final residual between consecutive refinement levels:")
        resids = completed["final_residual"].dropna().values
        for i in range(1, len(resids)):
            rel_change = abs(resids[i] - resids[i - 1]) / (abs(resids[i - 1]) + 1e-30)
            icon = "✓" if rel_change < 0.05 else "✗"
            print(f"  {icon}  levels {i-1}→{i}: {rel_change*100:.2f}% change")